<a href="https://colab.research.google.com/github/lizaoh/smp_program_data/blob/main/smp2017_extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top of Script

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install pymupdf
!pip install pymupdf-layout
!pip install pymupdf4llm
!pip install wordfreq
# !pip install rapidfuzz
import glob
import os
import pathlib
import pymupdf
import pymupdf.layout
import pymupdf4llm
import re
import pandas as pd
import unicodedata
import wordfreq
# from rapidfuzz import process, fuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: PyMuPDF
    Found existing installation: PyMuPDF 1.26.7
    Uninstalling PyMuPDF-1.26.7:
      Successfully uninstalled PyMuPDF-1.26.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.1/183.1 kB 13.9 MB/s eta 0:00:00


In [5]:
pdfs_path = '/content/drive/MyDrive/math_psych_work/Conference Programs/'

# Functions
Created with help from GPT 5.2, but some are my own code just turned into a function.

In [173]:
LOCATIONS = [
    'United States of America', 'United States', 'Switzerland', 'Japan', 'Bremen',
    'Berlin, Germany', 'Heidelberg, Germany', 'Germany', 'Berlin', 'Norway',
    'Turkey', 'Belgium', 'Italy', 'Israel', 'New Brunswick, New Jersey',
    'Australia', 'The Netherlands', 'USA', 'Netherlands, The', 'Netherlands',
    'United Kingdom', 'Singapore', 'France', 'Dayton OH', 'Dayton', 'India',
    'Taiwan, Republic of China', 'Austria', 'Canada', 'Denmark', 'Spain',
    'Edmonton', 'Bloomington, Indiana', 'Indiana', 'Russian Federation',
    'University Park, Pennsylvania', 'California', 'San Francisco, California',
    'Taipei, Taiwan', 'Charlottesville, Virginia', 'New York, New York',
    'Toronto, Ontario', 'New Haven, Connecticut', 'Ann Arbor, Michigan', 'Ohio',
    'Ottawa, Ontario', 'Houston, Texas', 'UK', 'New Brunswick, Piscataway, NJ',
    'Finland'
]

# compile once
LOCATION_RE = re.compile(
    r',\s*(?:' + '|'.join(map(re.escape, LOCATIONS)) + r')\b',
    re.I
)

def remove_locations(entry: str) -> str:
    return LOCATION_RE.sub('', entry).strip()

In [7]:
# Authors here are written normally so no comma
AUTHOR_LINE_RE = re.compile(
    r"""
    ^
    # First author
    (?:[a-z]{2,}\s+)*[A-Z][a-zA-Z'-]+,      # last name (with lowercase particles)
    \s+
    (?:[A-Z][a-zA-Z'-]+|[A-Z]\.)            # first name OR initial
    (?:\s+(?:[A-Z][a-zA-Z'-]+|[A-Z]\.))*    # middle names / initials

    # Additional authors
    (?:,\s+
        (?:[a-z]{2,}\s+)*[A-Z][a-zA-Z'-]+,
        \s+
        (?:[A-Z][a-zA-Z'-]+|[A-Z]\.)
        (?:\s+(?:[A-Z][a-zA-Z'-]+|[A-Z]\.))*
    )*

    # Optional ", and Last, First"
    (?:,\s+and\s+
        (?:[a-z]{2,}\s+)*[A-Z][a-zA-Z'-]+,
        \s+
        (?:[A-Z][a-zA-Z'-]+|[A-Z]\.)
        (?:\s+(?:[A-Z][a-zA-Z'-]+|[A-Z]\.))*
    )?

    \.?
    $
    """,
    re.VERBOSE
)

def is_author_only_line(line: str) -> bool:
    return bool(AUTHOR_LINE_RE.match(line.strip()))

In [197]:
def clean_authors(text: str) -> str:
    if not text:
        return text

    TEXT_TO_DELETE = [
        r'\[\d{2}/\d{2}\s+\d{2}:\d{2}\]\s*Tiered',  # [24/07 11:20] Tiered
        r'##',
        r'\bThe Slate\b',
        r'Space\s+\d{2}\s+Scarman',
        r'Tiered\s+Scarman',
    ]

    for pattern in TEXT_TO_DELETE:
        text = re.sub(pattern, '', text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [156]:
def parse_authors_with_indices(auth_text: str):
    authors = []

    for name, nums in re.findall(r'([^,\[]+)\[([0-9,]+)\]', auth_text):
        indices = [int(n) for n in nums.split(',')]
        authors.append((name.strip().lstrip('and '), indices))

    return authors

In [198]:
def normalize_affiliation_numbers(text: str) -> str:
    # Turn [3] → 3
    return re.sub(r'\[(\d+)\]', r'\1', text)

In [199]:
def parse_affiliation_dict(aff_text: str) -> dict[int, str]:
    aff_text = normalize_affiliation_numbers(aff_text)

    out = {}

    # Pattern A: number FIRST
    for m in re.finditer(r'(\d+)\s+([^0-9]+?)(?=(?:\s*\d+\s+)|$)', aff_text):
        num = int(m.group(1))
        aff = m.group(2).strip(' ,')
        out[num] = aff

    # Pattern B: number LAST
    for m in re.finditer(r'([^0-9]+?)\s*(\d+)(?=\s*(?:,|$))', aff_text):
        num = int(m.group(2))
        aff = m.group(1).strip(' ,')
        out[num] = aff

    return out

In [206]:
def make_aff_list(authors, aff_dict):
    author_names = []
    author_affiliations = []

    for name, indices in authors:
        author_names.append(name)

        affs = [
            aff_dict[i]
            for i in indices
            if i in aff_dict
        ]

        # join multiple affiliations for the SAME author with " and "
        author_affiliations.append(" / ".join(affs))

    new_authors = ", ".join(author_names)
    new_affiliations = "; ".join(author_affiliations)

    return new_authors, new_affiliations

In [207]:
def rehyphenate_words(text, words_to_hyphenate):
    for word, hyphenated_word in words_to_hyphenate:
        text = text.replace(word, hyphenated_word)

    return text

In [208]:
def remove_trailing_text(text):
  no_trailing_junk = text.split(".")[:-1]

  return ".".join(no_trailing_junk) + "."

In [210]:
def remove_schedule_noise(text: str) -> str:
    if not text:
        return text

    # 1. Remove date/time/location lines
    text = re.sub(
        r'''
        \n*                             # leading newlines
        \d{2}/\d{2}                    # date
        \s+\d{2}:\d{2}                 # time
        [^\n]*                         # rest of line (location)
        \n*                            # trailing newlines
        ''',
        '\n\n',
        text,
        flags=re.VERBOSE
    )

    # 2. Remove standalone page numbers (e.g. "3", "129")
    text = re.sub(
        r'\n\s*\d{1,3}\s*\n',
        '\n\n',
        text
    )

    # 3. Normalize whitespace
    text = re.sub(r'\n{2,}', '\n\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return text.strip()

In [211]:
def remove_title_noise(text: str) -> str:
  # Remove text interrupting the title
  text = re.sub(
    r'\*\*\s+- \d{2}/\d{2} \d{2}:\d{2}\s+\*\*',
    ' ',
    text)

  text = re.sub(
      r'\*\* \d{2}/\d{2} \d{2}:\d{2} \*\*',
      ' ',
      text
      )

  abstracts = text.replace('** \n\n## **', ' ')

  return abstracts

In [212]:
def clean_text(text, words_to_hyphenate=None):
    if not text:
        return text

    if words_to_hyphenate:
      text = rehyphenate_words(text, words_to_hyphenate)

    text = remove_schedule_noise(text)
    text = remove_trailing_text(text)
    text = text.replace('\n\n[', '[') # Remove new lines for references
                                      # at the end of one abstract
    text = re.sub(r'\n\n\*\*==> picture [\d{3} x \d{2}] intentionally omitted <==\*\*\n\n', '', text)
    text = re.sub(r' \n\n\d{1,3} \n\n', ' ', text)  # Remove page breaks with page number
    text = re.sub(r'\n\n\d{2}', '', text)

    text = re.sub(r'-\s+(?!\b(?:and|or)\b)', '', text)  # Get rid of - and space after
                                                        # unless word after is
                                                        # "and" or "or"

    text = text.strip()
    text = fix_ligatures(text)

    return text

In [213]:
LIGATURE_MAP = {
    "ﬁ": "fi", "ﬂ": "fl", "ﬃ": "ffi", "ﬄ": "ffl", "ﬀ": "ff", "ﬅ": "ft", "ﬆ": "st",
    "Æ": "ffi", "¨u": "ü", "¨a": "ä", "´e": "é", "`e": "è", "`a": "à", "¨o": "ö",
    "˚a": "å", "c¸": "ç", '“': '"', '”': '"', "’": "'", '˜n': 'ñ', 'ˇs': 'š',
    "âĂŸ": "'", "``": '"', "↵": "ff", "✏": "ffl"
}

def fix_ligatures(text):
    # Replace known ligatures
    for bad, good in LIGATURE_MAP.items():
        text = text.replace(bad, good)

    # Replace any private-use ligature (common in PDFs)
    cleaned_chars = []
    for ch in text:
        name = unicodedata.name(ch, "")
        if "LIGATURE" in name.upper():
            # Try to break it apart: remove spaces and lowercase
            base = name.split("LIGATURE")[-1]
            base = base.replace(" ", "").lower()
            cleaned_chars.append(base)
        else:
            cleaned_chars.append(ch)

    return "".join(cleaned_chars)

In [214]:
# Checks if valid word using Zipf frequency
def is_probably_valid(word, threshold=2.5):
    return wordfreq.zipf_frequency(word, "en") > threshold  # smaller number cuts off
                                                            # more words, bigger is
                                                            # more lenient

# Program

241 entries total (3 plenary talks, 25 symposium talks, 140 talks, and 73 posters)

There are affiliations listed again here.


## Grab text from the pdf

In [215]:
year = '2017'
program = pymupdf.open(pdfs_path + f'smp{year}_program.pdf')

In [216]:
program_text = pymupdf4llm.to_markdown(program)

=== Document parser messages ===
                                    Full-page OCR on page.number=10/11.



In [232]:
program_text[80_600:83_000]

'uthor Index**<br>**133**|\n\n\n\nxvii \n\nxviii \n\n## **Abstracts** \n\n## **Some things that glitter are gold: A Shiny app for an Instance-Based Learning model** \n\nCleotilde Gonzalez, Je↵rey Stephen Chrabaszcz Carnegie Mellon University, United States of America \n\nDeparting from the fact that models are only a representation of reality and not reality itself, all models are wrong. Therefore, building models that are useful and finding ways to e↵ectively communicate their emerging insights are perhaps the most important goals for cognitive modelers. Unfortunately, we are terrible at getting a scientific message across to those less familiar with our models, and we often fall trap of technical complexity. The need for increased transparency, awareness and access to the insights that cognitive models can provide, has motivated the development of tools to give hands-on experiences with cognitive models. In this talk I will present our most recent attempt to make an IBL model useful 

## Split text into presentation entries

In [234]:
all_abstracts = program_text.split('**Abstracts** \n\n##')[1] # this is where abstracts start
# Mostly makes title one chunk again if broken up by other text
no_title_interruptions = remove_title_noise(all_abstracts)
split_abstracts = re.split(r'\n\n## \*\*', no_title_interruptions)
initial_abstract_entries = [entry.strip().lstrip('**') for entry in split_abstracts if entry.strip()][:-1]

# Splits up entries that may have multiple talks
abstract_entries = [
      part.strip()
      for entry in initial_abstract_entries
      for part in re.split(r'\*\*(?=[A-Z])', entry)
      if part.strip()
  ]
abstract_entries = [entry for entry in abstract_entries]

In [235]:
abstract_entries[:2]

['Some things that glitter are gold: A Shiny app for an Instance-Based Learning model** \n\nCleotilde Gonzalez, Je↵rey Stephen Chrabaszcz Carnegie Mellon University, United States of America \n\nDeparting from the fact that models are only a representation of reality and not reality itself, all models are wrong. Therefore, building models that are useful and finding ways to e↵ectively communicate their emerging insights are perhaps the most important goals for cognitive modelers. Unfortunately, we are terrible at getting a scientific message across to those less familiar with our models, and we often fall trap of technical complexity. The need for increased transparency, awareness and access to the insights that cognitive models can provide, has motivated the development of tools to give hands-on experiences with cognitive models. In this talk I will present our most recent attempt to make an IBL model useful to researchers and students of behavioral science. Shiny-IBL uses the R packa

## Find words to re-hyphenate

In [236]:
pattern = re.compile(r'([A-Za-z]+)-\n\s*([A-Za-z]+)')
possible_hyphenated_words = []

counter = 0
for p, page in enumerate(program[29:]):  # these are the pages with abstracts only
  text = fix_ligatures(page.get_text('text'))
  matches = pattern.findall(text)

  for left, right in matches:
    word = f"{left}{right}"
    hyphenated_word = f"{left}-{right}"
    if not is_probably_valid(word, threshold=2.8):
      possible_hyphenated_words.append([word, hyphenated_word])

      print(f"{counter:>3}: Page {p+2:<3} {hyphenated_word:<30} {word}")
      counter += 1

  0: Page 3   previously-observed            previouslyobserved
  1: Page 3   iden-tifiability               identifiability
  2: Page 6   agreement-attraction           agreementattraction
  3: Page 8   short-term                     shortterm
  4: Page 11  mind-wandering                 mindwandering
  5: Page 14  proof-of                       proofof
  6: Page 17  individual-trial               individualtrial
  7: Page 17  individual-trial               individualtrial
  8: Page 17  population-level               populationlevel
  9: Page 17  over-weighting                 overweighting
 10: Page 19  intact-rearranged              intactrearranged
 11: Page 19  un-derlies                     underlies
 12: Page 23  brain-imaging                  brainimaging
 13: Page 27  decision-making                decisionmaking
 14: Page 27  accuracy-stressed              accuracystressed
 15: Page 28  scale-invariant                scaleinvariant
 16: Page 30  parame-terized                

In [238]:
# Pick indices of words to rehyphenate
indices = [0, (2, 10), (12, 15), 20, 21, 23, 24, 26, (29, 32), 38, 40, 43, 45, 47, (52, 54), 60, 62, (64, 66), 69, 80, 84, 85, 88, 89, 91, 92]
words_to_hyphenate = [
    possible_hyphenated_words[i]
    for item in indices
    for i in ([item] if isinstance(item, int) else range(*item))
]

In [239]:
for word, hyphenated_word in words_to_hyphenate:
  print(f'{word:<30} {hyphenated_word}')

previouslyobserved             previously-observed
agreementattraction            agreement-attraction
shortterm                      short-term
mindwandering                  mind-wandering
proofof                        proof-of
individualtrial                individual-trial
individualtrial                individual-trial
populationlevel                population-level
overweighting                  over-weighting
brainimaging                   brain-imaging
decisionmaking                 decision-making
accuracystressed               accuracy-stressed
testretest                     test-retest
relapserecovery                relapse-recovery
categorizationdecision         categorization-decision
QuasiHyperbolic                Quasi-Hyperbolic
attributebased                 attribute-based
crossmeasure                   cross-measure
tradeoffs                      trade-offs
nondynamical                   non-dynamical
factorialtechnology            factorial-technology
opensource   

## Sort authors, affiliations, title, and abstract

In [240]:
parsed_entries = []

for entry in abstract_entries:
  cleaned_entry = clean_text(entry, words_to_hyphenate)

  # Gets all parts of title inbetween bolding
  title_parts = re.findall(r'\*\*(.*?)\*\*', '**' + cleaned_entry)
  cleaned_title = ' '.join(a.strip() for a in title_parts) if title_parts else None

  rest_of_entry = cleaned_entry.split('**')[1].strip()
  # Splits into author, affiliation, abstract for most of them
  entry_parts = rest_of_entry.split('\n\n', 2)

  # If only 2 parts, then authors and affiliations are on the same line
  if len(entry_parts) == 2:
    auth_and_aff, abstract = entry_parts

    # Separates authors and affiliations
    auth_and_aff = remove_locations(auth_and_aff)
    auth_and_aff = auth_and_aff.replace('## ', '')
    # If multiple affiliations
    if ']' in auth_and_aff:
      authors, affiliations = re.split(r'(?<=\])\s+(?=1)', auth_and_aff, maxsplit=1)
    # If just one affiliation at the end, I'll fix manually
    else:
      authors = auth_and_aff
      affiliations = ''

  # Otherwise sort title, authors, affiliations, and abstract
  elif len(entry_parts) == 3:
    authors, affiliations, abstract = entry_parts


  authors = authors.replace(' and ', ' , ')
  # Turns affiliations into list of author's affiliations instead of just list of
  # possible affiliations (like in previous years' programs)
  if ']' in authors:
    parsed_authors = parse_authors_with_indices(authors)
    aff_dict = parse_affiliation_dict(affiliations)
    authors, affiliations = make_aff_list(parsed_authors, aff_dict)

  # Cleans up irrelevant text and extra spaces
  cleaned_authors = clean_authors(authors)

  cleaned_affiliations = remove_locations(affiliations)   # Gets rid of locations
                                                          # of affiliations
  cleaned_affiliations = cleaned_affiliations.strip().lstrip('## ')

  cleaned_abstract = re.sub(r'\s+', ' ', abstract.strip())
  cleaned_abstract = remove_trailing_text(cleaned_abstract)

  parsed_entries.append({
    'year': year,
    'author(s)': cleaned_authors,
    'affiliation(s)': cleaned_affiliations,
    'title': cleaned_title,
    'type': '',
    'abstract': cleaned_abstract
  })

In [241]:
parsed_entries[:2]

[{'year': '2017',
  'author(s)': 'Cleotilde Gonzalez, Jeffrey Stephen Chrabaszcz Carnegie Mellon University',
  'affiliation(s)': '',
  'title': 'Some things that glitter are gold: A Shiny app for an Instance-Based Learning model',
  'type': '',
  'abstract': 'Departing from the fact that models are only a representation of reality and not reality itself, all models are wrong. Therefore, building models that are useful and finding ways to effectively communicate their emerging insights are perhaps the most important goals for cognitive modelers. Unfortunately, we are terrible at getting a scientific message across to those less familiar with our models, and we often fall trap of technical complexity. The need for increased transparency, awareness and access to the insights that cognitive models can provide, has motivated the development of tools to give hands-on experiences with cognitive models. In this talk I will present our most recent attempt to make an IBL model useful to researc

# Create df and convert to csv

In [242]:
df = pd.DataFrame(parsed_entries, columns=["year", "author(s)", "affiliation(s)", "title", "type", "abstract"])

In [243]:
df.head()

,year,author(s),affiliation(s),title,type,abstract
0,2017,"Cleotilde Gonzalez, Jeffrey Stephen Chrabaszcz...",,Some things that glitter are gold: A Shiny app...,,Departing from the fact that models are only a...
1,2017,"Paul Mätzig, Shravan Vasishth, Felix Engelmann...",University of Potsdam; University of Potsdam; ...,A computational investigation of sources of va...,,We present a computational evaluation of three...
2,2017,"Roger Ratcliff, Gail McKoon",Ohio State University,Modeling Decision Processes on a Continuous Scale,,I present a model for perceptual decision maki...
3,2017,"Mikhail Spektor, David Kellen",University of Basel; Syracuse University,You can't make a silk purse of a sow's ear: On...,,Formal modeling approaches to cognition provid...
4,2017,Elliot Andrew Ludvig,University of Warwick,Modeling memory biases in decisions from exper...,,When people make decisions based on past exper...


In [244]:
df.to_csv(f"/content/drive/MyDrive/math_psych_work/csv/smp{year}_program.csv", index=False)